In [ ]:
import numpy as np
import xarray as xr
from matplotlib import pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = (8,5)
from scipy.interpolate import *

In [ ]:
x = np.linspace(0, 2*np.pi, 11)[0:-1]
y = np.sin(x)

# analytic derivative:
dydxa = np.cos(x)

# numpy derivatives:
dydx1 = np.diff(y)/np.diff(x)
np.shape(y),np.shape(dydx1)

In [ ]:
# make a new grid for the derivative at half points: (numpy does not associate vars and coords)
h = x[1] - x[0]
x_half = x[0:-1] + h / 2

In [ ]:
plt.plot(x,y,marker='o',label='y')
plt.plot(x,dydxa,marker='o',label='analytic dydx')
plt.plot(x[0:-1],dydx1,marker='o',label='numpy dydx - left points')
plt.plot(x[1:],dydx1,marker='o',label='numpy dydx - right points')
plt.plot(x_half,dydx1,marker='o',label='numpy dydx on half points')
plt.legend(loc='lower left')

In [ ]:
# xarray derivatives:
da = xr.DataArray(y ,
                  dims=['x'],
                  coords={'x': x})

dydxDA = da.diff('x')/da.x.diff('x')
dydxDA.x

In [ ]:
plt.plot(x,dydxa,marker='o',label='analytic dydx')
dydxDA.plot(marker='o',label='xarray?') # default coordinates?
plt.legend()

In [ ]:
dy = da.diff('x').rolling(x=2).mean().shift(x=-1).dropna('x')
dx = da.x.diff('x').rolling(x=2).mean().shift(x=-1).dropna('x')
dydx = dy/dx

plt.plot(x,dydxa,marker='o',label='analytic dydx')
dydx.plot(marker='o',label='diff+interp+shift') # default coordinates?
plt.legend()

In [ ]:
# What about boundary points?  We could extend the whole DataArray:
da_extend = xr.concat([da[-1], da, da[0]],'x')
da_extend

In [ ]:
# ... but the coordinate variable gets messed up - must separately extend the data and coordinate
y_extend = xr.concat([da[-1], da, da[0]],'x')
x_extend = xr.concat([da.x[0]-h, da.x, da.x[-1]+h],'x')

In [ ]:
dy = y_extend.diff('x').rolling(x=2).mean().shift(x=-1).dropna('x')
dx = x_extend.diff('x').rolling(x=2).mean().shift(x=-1).dropna('x')
dydx_extended = dy/dx

In [ ]:
plt.plot(x,dydxa,marker='o',label='analytic dydx')
dydx_extended.plot(marker='o',label='second order dydx')
plt.plot(x_half,dydx1,marker='o',label='numpy dydx')
plt.legend()

In [ ]:
abs(dydxa-dydx_extended).max()

In [ ]:
for numpoints in [11,21,41,81,161]:
    x = np.linspace(0, 2*np.pi, numpoints)[0:-1]
    y = np.sin(x)
    dydx = np.cos(x)
    da = xr.DataArray(y , dims=['x'], coords={'x': x})
    h = x[1] - x[0]
    y_extend = xr.concat([da[-1], da, da[0]],'x')
    x_extend = xr.concat([da.x[0]-h, da.x, da.x[-1]+h],'x')
    dy = y_extend.diff('x').rolling(x=2).mean().shift(x=-1).dropna('x')
    dx = x_extend.diff('x').rolling(x=2).mean().shift(x=-1).dropna('x')
    residual = dy/dx - dydx
    print(abs(residual).max())
    residual.plot(label=numpoints)
plt.legend()